In [1]:
import pandas as pd
import numpy as np

from transactions import etl

trades = etl.Commsec().df
div_scrip = etl.DivScrip().df

In [2]:
# Get scrip
# Convert string dates to datetime.date
# div_scrip.date = pd.to_datetime(div_scrip.index, dayfirst=True).dt.date
div_scrip[div_scrip.Scrip.isna() == False]

,,Cash_unfranked,Cash_franked,Scrip,Scrip_price
Date,Ticker,,,,
2016-09-28,QBE,NaN,NaN,11.0,9.54
2016-10-07,WOW,NaN,NaN,1.0,33.00
2017-04-07,WOW,NaN,NaN,1.0,34.34
2017-04-13,QBE,NaN,NaN,13.0,12.97
2017-09-29,QBE,NaN,NaN,11.0,10.01
2017-10-06,WOW,NaN,NaN,2.0,24.86
2018-04-06,WOW,NaN,NaN,2.0,26.63
2018-04-20,QBE,NaN,NaN,2.0,9.89
2018-10-05,QBE,NaN,NaN,11.0,10.74


In [3]:
# Add scrip as vol and scrip_price as tradeprice, effectiveprice
xscrip_df = div_scrip[div_scrip.Scrip.isna() == False].drop(columns=['Cash_unfranked', 'Cash_franked'])

trades = pd.concat([trades, xscrip_df], sort=False)

trades

Market TradeType    Volume  TradePrice  EffectivePrice  \
Date       Ticker                                                          
2020-01-30 NEA       ASX         B     606.0       1.650        1.666502   
2020-01-16 IBXO      ASX         B  100099.0       0.010        0.010100   
2020-01-14 DEM       ASX         B    4170.0       0.240        0.242398   
           IFN       ASX         S    1767.0       0.680        0.668710   
2020-01-10 AVH       ASX         S    1551.0       0.645        0.638556   
...                  ...       ...       ...         ...             ...   
2017-10-06 WOW       NaN       NaN       NaN         NaN             NaN   
2018-04-06 WOW       NaN       NaN       NaN         NaN             NaN   
2018-04-20 QBE       NaN       NaN       NaN         NaN             NaN   
2018-10-05 QBE       NaN       NaN       NaN         NaN             NaN   
2018-10-12 WOW       NaN       NaN       NaN         NaN             NaN   

                   Brokerage  Scrip  Scrip_price  
Date       Ticker                                 
2020-01-30 NEA          10.0    NaN          NaN  
2020-01-16 IBXO         10.0    NaN          NaN  
2020-01-14 DEM          10.0    NaN          NaN  
           IFN          20.0    NaN          NaN  
2020-01-10 AVH          10.0    NaN          NaN  
...                      ...    ...          ...  
2017-10-06 WOW           NaN    2.0        24.86  
2018-04-06 WOW           NaN    2.0        26.63  
2018-04-20 QBE           NaN    2.0         9.89  
2018-10-05 QBE           NaN   11.0        10.74  
2018-10-12 WOW           NaN    2.0        31.80  

[77 rows x 8 columns]

In [4]:
import yfinance as yf
import datetime

tickers = sorted(set(trades.index._levels[1].to_list()))
for i, ticker in enumerate(tickers):
    # if ticker != 'IMM':
    #     continue
    
    # Refresh lookup masks
    mask_ticker = (trades.index.get_level_values(1)==ticker)
    mask_scrip = (trades['Brokerage'].isna()==True)

    # Store first date of holdings
    holding_start = trades.loc[mask_ticker,'Market'].first_valid_index()[0]
    
    print(f'\r{ticker} | Progress: {i+1}/{len(tickers)}', end='', flush=True)
    if len(trades.loc[trades['Brokerage'].isna()==True]) > 0:
        # Fill df with scrip data
        trades.loc[mask_scrip,'TradeType'] = 'B'
        trades.loc[mask_scrip,'Market'] = 'ASX'
        trades.loc[mask_scrip,'Volume'] = trades.loc[mask_scrip].pop('Scrip')
        trades.loc[mask_scrip,'TradePrice'] = trades.loc[mask_scrip].pop('Scrip_price')
        trades.loc[mask_scrip,'EffectivePrice'] = trades.loc[mask_scrip].pop('Scrip_price')
        trades.loc[mask_scrip,'Brokerage'] = 0

        trades = trades.drop(columns=['Scrip','Scrip_price'])

    # Update with stocksplit data
    try:
        splits = yf.Ticker(f'{ticker}.AX').splits
    except:
        continue
    else:
        if len(splits) == 0:
            continue
        
        if len(splits.loc[holding_start:]) > 0:
            split_multiple = splits.loc[holding_start:].cumprod().values[-1]
            trades.loc[mask_ticker,'Volume'] = trades.loc[mask_ticker].pop('Volume') * split_multiple
            trades.loc[mask_ticker,'TradePrice'] = trades.loc[mask_ticker].pop('TradePrice') / split_multiple
            trades.loc[mask_ticker,'EffectivePrice'] = trades.loc[mask_ticker].pop('EffectivePrice') / split_multiple

trades = trades.sort_index()

# Save as csv
today = datetime.datetime.today().date()

trades.to_csv(f'data/{today}_transactions.csv')

IBXO | Progress: 10/35- IBXO.AX: 1d data not available for startTime=-2208988800 and endTime=1581774421. Only 100 years worth of day granularity data are allowed to be fetched per request.
SRS | Progress: 30/35- SRS.AX: No data found, symbol may be delisted
ZNO | Progress: 35/35